# Algorithms... are regularly used for performance benchmarks

## Loops are of most importance in Finance

In [1]:
import random
def average_py(n):
    s = 0
    for i in range(n):
        s += random.random()
    return s / n

In [ ]:
n = 100000000

In [ ]:
%time average_py(n)

In [ ]:
# Times the function several times for a more reliable estimate
%timeit average_py(n) 

In [ ]:
# Alt. uses list comprehension instead of a function
%time sum([random.random() for _ in range(n)]) / n

### Vectorization with Numpy

#### Note to self -- It is tempting to wrie vectorized code with NumPy whenever possible due to concise syntax and speed improvements typically observed. However, these benifits often come at the proce of a much higher memory footprint.

In [ ]:
import numpy as np

In [ ]:
def average_np(n):
    s = np.random.random(n)
    return s.mean()

In [ ]:
%time average_np(n)

In [ ]:
%timeit average_np(n)

### Numba 

#### Numba (numba.pydata.org) is a package that allows the dynamic compiling of pure Python code by the use of LLVM. 

In [ ]:
import numba

In [ ]:
average_nb = numba.jit(average_py)
%time average_nb(n)

In [ ]:
# Second execution should be much faster
%time average_nb(n)

In [ ]:
# Very good average because the code was compiled one and then reused
%timeit average_nb(n)

### Cython

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a
import random 

def average_cy1(int n):
    cdef int i
    cdef float s = 0
    for i in range(n):
        s += random.random()
    return s / n
    

In [ ]:
%time average_cy1(n)

In [ ]:
%timeit average_cy1(n)

In [ ]:
%%cython 
from libc.stdlib cimport rand
cdef extern from 'limits.h':
    int INT_MAX
cdef int i
cdef float rn
for i in range(5):
    rn = rand() / INT_MAX
    print(rn)

In [ ]:
%%cython -a
from libc.stdlib cimport rand
cdef extern from 'limits.h':
    int INT_MAX

def average_cy2(int n):
    cdef int i 
    cdef float s = 0
    for i in range(n):
        s += rand() / INT_MAX
    return s / n

In [ ]:
%time average_cy2(n)

In [ ]:
%timeit average_cy2(n)

### Prime numbers factorization

In [ ]:
# Base case
def is_prime(I):
    if I % 2 == 0:
        return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0:
            return False
    return True

In [ ]:
n = int(1e8 + 3)

In [ ]:
n

In [ ]:
%time is_prime(n)

In [ ]:
p1 = int(1e8 + 7)

In [ ]:
p1

In [ ]:
%time is_prime(p1)

In [ ]:
p2 = 100109100129162907

In [ ]:
p2.bit_length()

In [ ]:
%time is_prime(p2)

### Numba

In [ ]:
is_prime_nb = numba.jit(is_prime)

In [ ]:
%time is_prime_nb(n)

In [ ]:
%time is_prime_nb(n)

In [ ]:
%time is_prime_nb(p1)

In [ ]:
%time is_prime_nb(p1)

In [ ]:
%time is_prime_nb(p2)

In [ ]:
%time is_prime_nb(p2)

### Cython 

In [ ]:
%%cython 

def is_prime_cy1(I):
    if I % 2 == 0:
        return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0:
            return False
    return True

In [ ]:
%timeit is_prime(p1)

In [ ]:
%timeit is_prime_cy1(p1)

In [ ]:
%%cython 

def is_prime_cy2(long I):
    cdef long i
    if I % 2 == 0:
        return False
    for i in range(3, int(I ** 0.5) +1, 2):
        if I % i == 0:
            return False
    return True

In [ ]:
%timeit is_prime_cy2(p1)

In [ ]:
%time is_prime_nb(p2)

In [ ]:
%time is_prime_cy2(p2)

### Multiplocessing 

In [ ]:
import multiprocessing as mp

In [ ]:
pool = mp.Pool(processes=4)

In [ ]:
%time pool.map(is_prime, 10 * [p1])

In [ ]:
%time pool.map(is_prime_nb, 10 * [p2])

In [ ]:
%time pool.map(is_prime_cy2, 10 * [p2])

### Fibonacci Numbers

#### Recursive

In [ ]:
def fib_rec_py1(n):
    if n < 2:
        return n
    else:
        return fib_rec_py1(n-1) + fib_rec_py1(n-2)


In [ ]:
%time fib_rec_py1(35)

In [ ]:
fib_rec_nb = numba.jit(fib_rec_py1)

In [ ]:
%time fib_rec_nb(35)

In [ ]:
%%cython
def fib_rec_cy(int n):
    if n < 2:
        return n
    else:
        return fib_rec_cy(n-1) + fib_rec_cy(n-2)
    

In [ ]:
%time fib_rec_cy(35)

#### The major problem with recursice algorithm is that intermediate results are not cached but rather recalculated. To avoid this particular problem, a decorator can be used that takes care of the caching of the intermediate results. This speeds up the execution by multiple orders of magnitude:

In [ ]:
from functools import lru_cache as cache

In [ ]:
@cache(maxsize=None)
def fib_rec_py2(n):
    if n < 2:
        return n 
    else:
        return fib_rec_py2(n-1) + fib_rec_py2(n-2)

In [ ]:
%time fib_rec_py2(35)

In [ ]:
%time fib_rec_py2(80)

#### Iterative

In [ ]:
def fib_it_py(n):
    x, y = 0, 1
    for i in range(1, n + 1):
        x, y = y, x + y 
        return x

In [ ]:
%time fib_it_py(80)

In [ ]:
fib_it_nb = numba.jit(fib_it_py)

In [ ]:
%time fib_it_nb(80)

In [ ]:
# not sure this is the expected behavior --> draft and send email to the publisher

## The Number Pi

In [ ]:
import random 
import numpy as np
from pylab import mpl, plt
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
rn = [(random.random() * 2 - 1, random.random() * 2 - 1) for _ in range(500)]

In [ ]:
rn = np.array(rn)

In [ ]:
rn[:5]

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(1,1,1)
circ = plt.Circle((0,0), radius=1, edgecolor='g', lw=2.0, facecolor='None')
box = plt.Rectangle((-1, -1), 2, 2, edgecolor='b', alpha=0.3)
ax.add_patch(circ)
ax.add_patch(box)
plt.plot(rn[:, 0], rn[:, 1], 'r.')
plt.ylim(-1.1, 1.1)
plt.xlim(-1.1, 1.1)
plt.show()


A NumPy implementation of this algorithm is rather conscise but also memory-intensive. Total execution time given the parameterization is about one second:

In [ ]:
n = int(1e7)

In [ ]:
%time rn = np.random.random((n, 2)) * 2 -1

In [ ]:
rn.nbytes

In [ ]:
%time distance=np.sqrt((rn ** 2).sum(axis=1))
distance[:8].round(10)

In [ ]:
%time frac = (distance <= 1.0).sum() / len(distance)

In [ ]:
pi_mcs = frac * 4
pi_mcs

Function below uses a for loop and implement Monte Carlo simulation in a memory-efficient manner. Note that the random numbers are not scaled in this case. The execution time is longer than teh NumPy version, but the Numba version is faster than NumPy:

In [ ]:
def mcs_pi_py(n):
    circle = 0
    for _ in range(n):
        x, y = random.random(), random.random()
        if (x ** 2 + y ** 2) ** 0.5 <= 1:
            circle += 1
    return (4 * circle) / n

In [ ]:
%time mcs_pi_py(n)

In [ ]:
import numba
mcs_pi_nb = numba.jit(mcs_pi_py)

In [ ]:
%time mcs_pi_nb(n)

A plain Cython version with static type declaration only does not perform that much faster than the Python version. However, relying again on the random number generation capabilities of C further speeds up the calculation considerably:

In [ ]:
%reload_ext Cython

In [ ]:
%%cython -a
import random 

def mcs_pi_cy1(int n):
    cdef int i, circle = 0
    cdef float x, y
    for i in range(n):
        x, y = random.random(), random.random()
        if (x ** 2 + y ** 2) ** 0.5 <= 1:
            circle += 1
    return (4 * circle) / n

In [ ]:
%time mcs_pi_cy1(n)

In [ ]:
%%cython -a
from libc.stdlib cimport rand
cdef extern from 'limits.h':
    int INT_MAX
def mcs_pi_cy2(int n):
    cdef int i, circle = 0
    cdef float x, y
    for i in range(n):
        x, y = rand() / INT_MAX, rand() / INT_MAX
        if (x ** 2 + y ** 2) ** 0.5 <= 1:
            circle += 1
    return (4 * circle) / n

In [ ]:
%time mcs_pi_cy2(n)

### Binomial Trees

In [ ]:
import math

In [ ]:
S0 = 36.
T = 1.0
r = 0.06
sigma = 0.2

In [ ]:
def simulate_tree(M):
    dt = T / M 
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u
    S = np.zeros((M + 1, M + 1))
    S[0,0] = S0
    z = 1
    for t in range(1, M + 1):
        for i in range(z):
            S[i, t] = S[i, t-1] * u
            S[i+1, t] = S[i, t-1] * d
        z +=1
    return S

In [ ]:
np.set_printoptions(formatter={'float':
                              lambda x: '%6.2f' % x})

In [ ]:
simulate_tree(4)

In [ ]:
%time simulate_tree(500)

#### NumPy

In [ ]:
M = 4

In [ ]:
up = np.arange(M + 1)
up = np.resize(up, (M + 1, M + 1))
up 

In [ ]:
down = up.T * 2
down 

In [ ]:
up - down 

In [ ]:
dt = T / M

In [ ]:
S0 * np.exp(sigma * math.sqrt(dt) * (up - down))

In [ ]:
def simulate_tree_np(M):
    dt = T / M
    up = np.arange(M + 1)
    up = np.resize(up, (M + 1, M + 1))
    down = up.transpose() * 2
    S = S0 * np.exp(sigma * math.sqrt(dt) * (up - down))
    return S

In [ ]:
simulate_tree_np(4)

In [ ]:
%time simulate_tree_np(500)

#### Numba

NOte: financial algorithm should be well suited to optimization through Numba dynamic compilation. And indeed, another speedup compared to the NumPy version of an order of magnitude is pbserved -->

In [ ]:
simulate_tree_nb = numba.jit(simulate_tree)

In [ ]:
simulate_tree_nb(4)

In [ ]:
%time simulate_tree_nb(500)

In [ ]:
%timeit simulate_tree_nb(500)

### Cython 

In [ ]:
%%cython -a
import numpy as np
cimport cython
from libc.math cimport exp, sqrt

cdef float S0 = 36.
cdef float T = 1.0
cdef float r = 0.06
cdef float sigma = 0.2

def simulate_tree_cy(int M):
    cdef int z, t, i
    cdef float dt, u, d
    cdef float[:, :] S = np.zeros((M + 1, M + 1), dtype=np.float32)
    dt = T / M
    u = exp(sigma * sqrt(dt))
    d = 1 / u
    S[0, 0] = S0
    z = 1
    for t in range(1, M + 1): 
        for i in range(z):
            S[i, t] = S[i, t-1] * u
            S[i+1, t] = S[i, i-1] * d
        z += 1
    return np.array(S)
    

In [ ]:
simulate_tree_cy(4)

In [ ]:
%time simulate_tree_cy(500)

## Monte Carlo Simulation

### Python MC

This sets the benchmark. Based on the simulation, a European put option is valued:

In [ ]:
import numpy as np
import 

In [ ]:
M = 100 # The number of time intervals for discretization
I = 50000 # The number of paths to be simulated

In [ ]:
def mcs_simulation_py(p):
    M, I = p
    dt = I / M
    S = np.zeros((M + 1, I))
   # S[0] = S0
    rn = np.random.standard_normal(S.shape) # The random numbers drawn in a single vectorized step
    for t in range(1, M + 1): # Nested loop for simulation based on Euler scheme
        for i in range(I):
            S[t, i] = S[t-1, i] * math.exp((r - sigma ** 2 / 2) * dt +
                                          sigma * math.sqrt(dt) * rn[t, i])
            return S
    

In [ ]:
time S = mcs_simulation_py((M, I))